# Notes


### Highway entry/exit
- Allgemein: <br>
    - Wenn anderes Auto neben ego ist, dann wird es eigetnlich nur schräg vor/hinter ego angezeigt und nicht parallel
    - Code refactoring
    - doc strings updaten
    - Kommentare anpassen
- dual exit/entry <br>
    - degree and col_distances anpassen, ggf. 2 unterschiedliche shapes für duals berücksichtigen
    - und runtern fahren / rauf fahren 
- single exit/entry und exit&entry
    - degree and col_distances anpassen
- auf exit abfahren / entry rauffahren:
    - mit exit noch mal checken

- Problem Kreuzungen:
    - Tankestelle und drum herum
    - double entry mit Ampel 
        - Weiter mit Situation, dass entry von links kommt
        - und es muss gefixt werden, dass ego_on_highway nicht erneuert wird, wenn ego auf highway_junction ist 

- Server
    - Fps fixen
    - UE Ordner überall pullen und Path anpassen




In [1]:
# TODO
 
# NEU
# -------------------------------------------------------------------------------------
 # 1) Simplify everything   
 # 2) config schreiben
 # 3) Doc strings (später)
 
# JP 
#--------------------------------------------------------------------------------------
 # 1) Highway Ampel (JP)
        # wenn rauf/runter 
 # 2) Finetuning:
        # Eine entry&exit ist vor mir, danach nicht neue auffahrt/Abfahrt, dann nach dem ich
        # von der junction runter bin, springt er auf normal road statt dynamisch rausfahren 
 # -----------------------------------------------------------------------------------


# ICH
#------------------------------------------------------------------------------------
 # 1) exitst
 # 2) Stadt Scenario testen 

# Initialize

In [2]:
import pygame
import carla
import random
import time
import math
import collections
from collections import Counter
import pandas as pd
import itertools
from copy import deepcopy
import warnings
import numpy as np
import random


warnings.filterwarnings("ignore")
from utils import *

pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
distance_to_junc = 20
direction_angle = 20
radius = 100

actor_list = []

for actor in actor_list:
    actor.destroy()
actor_list = []

client = carla.Client("localhost", 2000)
world = client.get_world()

blueprint_library = world.get_blueprint_library()
world_map = world.get_map()
spawn_points = world.get_map().get_spawn_points()
print(world.get_actors())

[Actor(id=24, type=spectator), Actor(id=25, type=traffic.stop), Actor(id=26, type=traffic.stop), Actor(id=27, type=traffic.stop), Actor(id=28, type=traffic.stop), Actor(id=29, type=traffic.stop), Actor(id=30, type=traffic.stop), Actor(id=31, type=traffic.stop), Actor(id=32, type=traffic.stop), Actor(id=33, type=traffic.unknown), Actor(id=34, type=traffic.unknown), Actor(id=35, type=traffic.yield), Actor(id=36, type=traffic.yield), Actor(id=37, type=traffic.yield), Actor(id=38, type=traffic.unknown), Actor(id=39, type=traffic.unknown), Actor(id=40, type=traffic.yield), Actor(id=41, type=traffic.speed_limit.40), Actor(id=42, type=traffic.speed_limit.40), Actor(id=43, type=traffic.speed_limit.90), Actor(id=44, type=traffic.speed_limit.90), Actor(id=45, type=traffic.speed_limit.90), Actor(id=46, type=traffic.speed_limit.90), Actor(id=47, type=traffic.speed_limit.90), Actor(id=48, type=traffic.speed_limit.90), Actor(id=49, type=traffic.speed_limit.90), Actor(id=50, type=traffic.speed_limit.

In [4]:
manual_car = world.get_actor(147)
ego_vehicle = world.get_actor(147)
second_vehicle = world.get_actor(153)
print(ego_vehicle.get_location())

Location(x=9.194640, y=154.451614, z=0.003704)


# New Code

In [5]:
def check_ego_on_highway(ego_vehicle_location, road_lane_ids, world_map):
    waypoints = []
    ego_waypoint = world_map.get_waypoint(ego_vehicle_location)

    # TODO: nur kurz zum testen drin:
    if ego_waypoint.is_junction:
        junction = ego_waypoint.get_junction()
        if junction.id == 1368:
            return True
    ##########
    
    waypoints.append(ego_waypoint)
    if ego_waypoint.get_left_lane() is not None:
        waypoints.append(ego_waypoint.get_left_lane())
    if ego_waypoint.get_right_lane() is not None:
        waypoints.append(ego_waypoint.get_right_lane())
    for wp in waypoints:
        ego_vehilce_road_id = wp.road_id
        lanes = []
        for id in road_lane_ids:
            if str(ego_vehilce_road_id) == id.split("_")[0]:
                lanes.append(id.split("_")[1])
        lanes = [int(lane) for lane in lanes]
        if len(lanes) >= 6 or (
            sorted(lanes) == list(range(min(lanes), max(lanes) + 1)) and len(lanes) >= 3
        ):
            return True

    return False

In [6]:
def prepare_waypoints(closest_waypoint, wps, ego_waypoint):
    wps = remove_wps(closest_waypoint, wps)
    wps = mark_start_end(wps)
    wps = list(sum(wps, ()))
    wps = [wp for wp in wps if wp[0]]
    wps_grouped = clustering_algorithm(wps, 6)
    wps_grouped = remove_unnecessary_waypoints_in_group(wps_grouped)
    return wps_grouped

In [7]:
def remove_wps(closest_waypoint, wps):
    is_traffic_light_junction = False
    wps_on_ego_lane = []
    for group in wps:
        enough_distance = (
            distance(group[0].transform.location, group[1].transform.location) > 5
        )

        if (
            group[0].road_id == closest_waypoint.road_id
            or group[0].previous(10)[0].road_id
            == closest_waypoint.previous(10)[0].road_id
        ) and enough_distance:
            wps_on_ego_lane.append(group)
    exit_and_entry_wp = []
    for group in wps:
        enough_distance = (
            distance(group[0].transform.location, group[1].transform.location) > 3
        )
        for wp in wps_on_ego_lane:
            if (
                (distance(wp[0].transform.location, group[0].transform.location) < 7
                and enough_distance
                and group not in wps_on_ego_lane)
                or group[0].road_id in [1450, 1412, 1421, 1432, 1442] 
            ):
                if group[0].road_id in [1450, 1412, 1421, 1432, 1442]:
                    is_traffic_light_junction = True
                    exit_and_entry_wp.append((group[0], group[0]))
                else:    
                    exit_and_entry_wp.append(group)
                break
            elif (
                (distance(wp[1].transform.location, group[1].transform.location) < 7
                and enough_distance
                and group not in wps_on_ego_lane)
            ):
                exit_and_entry_wp.append(group)

    if is_traffic_light_junction:
        wps_on_ego_lane[0] = (wps_on_ego_lane[0][1], wps_on_ego_lane[0][0])
        wps_on_ego_lane[1] = (wps_on_ego_lane[1][1], wps_on_ego_lane[1][0])

    if is_traffic_light_junction and closest_waypoint.previous(2)[0].lane_id > 0:
        exit_and_entry_wp = exit_and_entry_wp[0:2]

    result = wps_on_ego_lane + exit_and_entry_wp
    return list(dict.fromkeys(result))


In [8]:
def get_exit_or_entry(wps, exit_or_entry, junction):
    junction_wps = junction.get_waypoints(carla.LaneType().Driving)
    first = []
    second = []

    # special case for junction 1368: traffic light junction on highway
    if junction.id == 1368:
        entry_road_ids = [1450, 1412, 1421, 1432, 1442]
        for start_wp, end_wp in junction_wps:
            if start_wp.road_id in entry_road_ids:
                first.append(start_wp)
                second.append(end_wp)
        return (first, second)  # first = start_wp, second = end_wp

    if exit_or_entry == "exit":
        order = ["end", "start"]
    elif exit_or_entry == "entry":
        order = ["start", "end"]

    for group in wps:
        if len(group) != 4:
            for wp in group:
                if wp[1] == order[0]:
                    first.append(wp[0])

    for first_wp in first:
        for wp_pair in junction_wps:
            if wp_pair[0].id == first_wp.id:
                second.append(wp_pair[1])
                break
            elif wp_pair[1].id == first_wp.id:
                second.append(wp_pair[0])
                break

    if not first or not second:
        print("Error in get_exit()")
        return None
    if exit_or_entry == "exit":
        return (second, first)  # first = end_wp, second = start_wp
    elif exit_or_entry == "entry":
        return (first, second)  # first = start_wp, second = end_wp

In [9]:
def update_matrix(
    world_map,
    ego_vehicle,
    ego_location,
    highway_shape,
    wps,
    matrix,
    ego_waypoint,
    junction,
    cars_on_entryExit,
    direction_angle,
    ghost=False,
    on_entry = False,
    is_2nd_update = False,
):
    if not is_2nd_update and highway_shape[1] == 3:
        matrix[list(matrix.keys())[5]] = [0, 0, 0, 0, 0, 0, 0, 0]

    matrix = list(matrix.items())
    exit_entry_found = False
    entry_wps, exit_wps = highway_shape[2], highway_shape[3]

    # TODO: put in a config file
    # set the angle threshold depending on curve type
    if highway_shape[0] == "entry_and_exit":
        degree = 30
    elif highway_shape[0] in ["single_entry", "single_exit"]:
        degree = 20
    elif highway_shape[0] in ["dual_entry"] or highway_shape[0] in ["dual_exit"]:
        degree = 18  # TODO: test and fine tuning of degree
    elif highway_shape[0] in [ "traffic_light_junction_entry_left", "traffic_light_junction_entry_right"]:
        degree = 30
    else:
        # just in case
        degree = 20

    # get right_lane_end_wp & right_lane_start_wp for dual entry & exit surrounding cars angle calculation
    if highway_shape[0] in ["dual_entry", "dual_exit"]:
        for wps_group in wps:
            if (len(wps_group) == 4) and (wps_group[0][1] == "end"):
                right_lane_end_wp = get_right_lane_wp(wps_group)
            elif (len(wps_group) == 4) and (wps_group[0][1] == "start"):
                right_lane_start_wp = get_right_lane_wp(wps_group)
    else:
        right_lane_end_wp = None
        right_lane_start_wp = None

    # First: Look behind ego
    exit_entry_found, col_entryExit, matrix = search_entry_or_exit(
        world_map,
        ego_location,
        "behind",
        exit_entry_found,
        matrix,
        highway_shape,
        wps,
        entry_wps,
        exit_wps,
        degree,
        junction.id,
        right_lane_end_wp,
        right_lane_start_wp,
    )
    # if not yet found, look in front
    if exit_entry_found == False:
        _, col_entryExit, matrix = search_entry_or_exit(
            world_map,
            ego_location,
            "before",
            exit_entry_found,
            matrix,
            highway_shape,
            wps,
            entry_wps,
            exit_wps,
            degree,
            junction.id,
            right_lane_end_wp,
            right_lane_start_wp,
        )

    # get road ids of: entry/exit, city road before and highway road after junction object
    entry_road_id = []
    exit_road_id = []
    entry_city_road = []
    exit_city_road = []
    entry_city_lane_id = []    
    exit_city_lane_id = []
    entry_highway_road = []
    exit_highway_road = []
    entry_highway_lane_id = []
    exit_highway_lane_id = []

    if entry_wps:
        for entry_wp in entry_wps[0]:
            entry_city_road.append(entry_wp.previous(3)[0].road_id)
            entry_city_lane_id.append(abs(entry_wp.previous(3)[0].lane_id))
            entry_road_id.append(entry_wp.road_id)
        for entry_wp in entry_wps[1]:
            entry_highway_road.append(entry_wp.next(3)[0].road_id)
            entry_highway_lane_id.append(abs(entry_wp.next(3)[0].lane_id))
    if exit_wps:
        for exit_wp in exit_wps[1]:
            exit_city_road.append(exit_wp.next(3)[0].road_id)
            exit_city_lane_id.append(abs(exit_wp.next(3)[0].road_id))
            exit_road_id.append(exit_wp.road_id)
        for exit_wp in exit_wps[0]:
            exit_highway_road.append(exit_wp.previous(3)[0].road_id)
            exit_highway_lane_id.append(abs(exit_wp.previous(3)[0].lane_id))


    # detect surrounding cars on exit/entry
    for car in cars_on_entryExit:
        col = calculate_position_in_matrix(
            ego_location,
            ego_vehicle,
            car,
            dict(matrix),
            world_map,
            ego_vehicle.get_velocity(),
            ghost,
        )
        if col is None:
            continue
        # matrix = list(matrix)
        if matrix:
            ego_already_in_matrix = False
            for key, row in matrix:
                if 1 in row:
                    ego_already_in_matrix = True

            if (ego_already_in_matrix and car.id == ego_vehicle.id):
                continue
            other_car_waypoint = world_map.get_waypoint(car.get_location())
            other_car_road_id = other_car_waypoint.road_id
            other_car_lane_id = other_car_waypoint.lane_id
            
            if other_car_road_id in entry_city_road + exit_city_road and not on_entry:
                continue
            distance_clostest_starting_waypoint = distance(get_clostest_starting_waypoint(junction.get_waypoints(carla.LaneType().Driving), car.get_location()).transform.location, car.get_location())
            if other_car_road_id in entry_city_road + exit_city_road:
                if distance_clostest_starting_waypoint > 30:
                    continue
                elif distance_clostest_starting_waypoint < 15:
                    row = 6
                else:
                    row = 7
                if highway_shape[0] in ["entry_and_exit"]:
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                elif highway_shape[0] in ["dual_entry"]:
                    entry_city_lane_id.sort()
                    # dual entry
                    if abs(other_car_lane_id) == entry_city_lane_id[0]:
                        col_entry = col_entryExit
                    elif abs(other_car_lane_id) == entry_city_lane_id[1]:
                        col_entry = col_entryExit + 1


                elif highway_shape[0] in ["dual_exit"]:
                    entry_city_lane_id.sort()
                    # dual exit
                    if abs(other_car_lane_id) == entry_city_lane_id[0]:
                        col_entry = col_entryExit +1
                    elif abs(other_car_lane_id) == entry_city_lane_id[1]:
                        col_entry = col_entryExit
                else:
                    col_entry = col_entryExit
                    col_exit = col_entryExit
            


                if entry_city_road == exit_city_road:
                    if is_junction_behind(other_car_waypoint, 40):
                        check_junction = get_junction_behind(other_car_waypoint, 40)
                        road_lane_ids = get_all_road_lane_ids(world_map)
                        if is_highway_junction(car, other_car_waypoint, check_junction, road_lane_ids, direction_angle):
                            on_exit_street = True
                        else:
                            on_exit_street = False
                else:
                    on_exit_street = False
                if (other_car_road_id in entry_city_road) and (col_entry < 8) and not on_exit_street:
                    insert_in_matrix(matrix, car, ego_vehicle, col_entry, row)
                elif (other_car_road_id in exit_city_road) and (col_exit < 8) and on_exit_street:
                    insert_in_matrix(matrix, car, ego_vehicle, col_exit, row)
                continue
                
            distance_junction_end = get_distance_junction_end(
                other_car_waypoint
            )
            distance_junction_start = get_distance_junction_start(
                other_car_waypoint
            )
            if other_car_road_id in entry_road_id + entry_highway_road:
                if highway_shape[0] == "dual_entry": 
                    yaw_difference = abs(
                        right_lane_end_wp.transform.rotation.yaw
                    ) - abs(
                        right_lane_end_wp.previous(distance_junction_end)[
                            0
                        ].transform.rotation.yaw
                    )
                else:
                    yaw_difference = abs(entry_wps[1][0].transform.rotation.yaw) - abs(
                        entry_wps[1][0]
                        .previous(distance_junction_end)[0]
                        .transform.rotation.yaw
                    )
            elif other_car_road_id in exit_road_id + exit_highway_road:
                if highway_shape[0] == "dual_exit":
                    yaw_difference = abs(
                        right_lane_start_wp.transform.rotation.yaw
                    ) - abs(
                        right_lane_start_wp.next(distance_junction_start)[
                            0
                        ].transform.rotation.yaw
                    )
                else:

                    yaw_difference = abs(exit_wps[0][0].transform.rotation.yaw) - abs(
                        exit_wps[0][0]
                        .next(distance_junction_start)[0]
                        .transform.rotation.yaw
                    )
             
            already_in_curve = abs(yaw_difference) > degree
            already_after_curve = abs(yaw_difference) > degree * 2
            if not on_entry:
                if already_after_curve:
                    row = 7
                elif already_in_curve:

                    row = 6
                else:
                    row = 5
            else:
                row = 5
                """
                distance_bounding_box = distance(junction.bounding_box.location, car.get_location())
                print(distance_bounding_box)
                if distance_bounding_box < 30:
                    row = 5
                if distance_bounding_box < 40:
                    row = 6
                else:
                    row = 7
                """
            if already_in_curve:
                if highway_shape[0] in ["entry_and_exit"]:
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                elif highway_shape[0] in ["dual_entry"]:
                    # dual entry
                    entry_road_id = [
                        x for _, x in sorted(zip(entry_highway_lane_id, entry_road_id))
                    ]
                    entry_highway_lane_id.sort()
                    if other_car_road_id == entry_road_id[0]:
                        col_entry = col_entryExit
                    elif other_car_road_id == entry_road_id[1]:
                        col_entry = col_entryExit + 1
                elif highway_shape[0] in ["dual_exit"]:
                    # dual exit
                    exit_road_id = [
                        x for _, x in sorted(zip(exit_highway_lane_id, exit_road_id))
                    ]
                    exit_highway_lane_id.sort()
                    if other_car_road_id == exit_road_id[0]:
                        col_exit = col_entryExit + 1
                    elif other_car_road_id == exit_road_id[1]:
                        col_exit = col_entryExit  # + 1
                else:
                    col_entry = col_entryExit
                    col_exit = col_entryExit
            else:
                if (highway_shape[1] == 3) and (col <= (col_entryExit + 1)):
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                else:
                    if highway_shape[0] in ["dual_entry"]:
                        # dual entry
                        entry_road_id = [
                            x
                            for _, x in sorted(
                                zip(entry_highway_lane_id, entry_road_id)
                            )
                        ]
                        entry_highway_lane_id.sort()
                        if other_car_road_id == entry_road_id[0]:
                            row = row - 1
                    elif highway_shape[0] in ["dual_exit"]:
                        # dual exit
                        exit_road_id = [
                            x
                            for _, x in sorted(zip(exit_highway_lane_id, exit_road_id))
                        ]
                        exit_highway_lane_id.sort()
                        if other_car_road_id == exit_road_id[0]:
                            row = row - 1
                    col_entry = col
                col_exit = col

            entry_streets = entry_road_id + entry_highway_road
            exit_streets = exit_road_id + exit_highway_road
            if (other_car_road_id in entry_streets) and (col_entry < 8):
                insert_in_matrix(matrix, car, ego_vehicle, col_entry, row)

            elif (other_car_road_id in exit_streets) and (col_exit < 8):
                insert_in_matrix(matrix, car, ego_vehicle, col_exit, row)

    return dict(matrix)

In [10]:
def search_entry_or_exit(
    world_map,
    ego_vehicle_location,
    direction,
    exit_entry_found,
    matrix,
    highway_shape,
    wps,
    entry_wps,
    exit_wps,
    degree,
    junction_id,
    right_lane_end_wp,
    right_lane_start_wp,
):
    ego_waypoint = world_map.get_waypoint(ego_vehicle_location)

    # for dual entry/exit we compare yaw values based on most right highway lane
    if highway_shape[0] in ["dual_exit", "dual_entry"]:
        initial_wp = ego_waypoint
        if direction == "before":
            while True:
                tmp_right_lane = initial_wp.get_right_lane()
                if (tmp_right_lane is None) or (
                    tmp_right_lane.lane_type != carla.LaneType().Driving
                ):
                    break
                initial_wp = tmp_right_lane
        # for looking behind, we consider left line first bc. otherwise wp.previous() go on entry
        elif direction == "behind":
            while True:
                tmp_left_lane = initial_wp.get_left_lane()
                if (tmp_left_lane is None) or (
                    tmp_left_lane.lane_type != carla.LaneType().Driving
                ):
                    break
                initial_wp = tmp_left_lane

    # for other entry/exit we compare yaw on entry/exit
    else:
        initial_wp = ego_waypoint

    col_entryExit = None
    # Look in front
    if direction == "before":
        z = 3  # column help in matrix
        col_distances = [6, 15, 35, 55, 75]
    # Look behind
    elif direction == "behind":
        z = (
            -1
        )  # column help in matrix; -1 because in list is first value only a threshold to see when junction too far behind
        if highway_shape[0] in [
            "entry_and_exit"
        ]:  # special because entry & exit are very close compared to normal column distance in matrix
            col_distances = [55, 20, 10, 6]
        elif highway_shape[0] in ["dual_exit"]:  # special because for dual exit
            col_distances = [55, 35, 20, 6]
        else:
            col_distances = [55, 35, 15, 6]
    for i, col_distance in enumerate(
        col_distances
    ):  # index 0 equals distance closest (10)
        if exit_entry_found:
            break
        for j in [
            -5,
            0,
            5,
        ]:  #  check 3 values in each column, bc. otherwise sometimes jumps are bigger than junction
            if exit_entry_found:
                break
            # Look in front
            if direction == "before":
                # if (col_distance == 4) and (j == 5):
                #     j = 0
                wp = initial_wp.next(col_distance + j)[0]
            # Look behind
            elif direction == "behind":
                if col_distance == 6 and j == -5:
                    continue
                # j = 0
                if highway_shape[0] == ["dual_entry", "dual_exit"]:
                    wp = initial_wp.previous(col_distance + j)[0]
                    while True:
                        tmp_right_lane = wp.get_right_lane()
                        if (tmp_right_lane is None) or (
                            tmp_right_lane.lane_type != carla.LaneType().Driving
                        ):
                            break
                        wp = tmp_right_lane
                else:
                    wp = initial_wp.previous(col_distance + j)[0]

            if (wp.is_junction) and (wp.get_junction().id == junction_id):
                # now we have to check if parallel or curve away, depending on exit / entry as first search
                if highway_shape[0] in [
                    "entry_and_exit",
                    "single_exit",
                ]:  # search for exit first
                    distance_junction_start = get_distance_junction_start(wp)
                    yaw_difference = abs(exit_wps[0][0].transform.rotation.yaw) - abs(
                        exit_wps[0][0]
                        .next(distance_junction_start)[0]
                        .transform.rotation.yaw
                    )
                    already_in_curve = abs(yaw_difference) > degree
                elif highway_shape[0] in ["single_entry"]:
                    distance_junction_end = get_distance_junction_end(wp)
                    yaw_difference = abs(entry_wps[1][0].transform.rotation.yaw) - abs(
                        entry_wps[1][0]
                        .previous(distance_junction_end)[0]
                        .transform.rotation.yaw
                    )
                    already_in_curve = abs(yaw_difference) < degree
                elif highway_shape[0] in ["dual_entry"]:
                    yaw_difference = abs(
                        right_lane_end_wp.transform.rotation.yaw
                    ) - abs(wp.transform.rotation.yaw)
                    already_in_curve = abs(yaw_difference) < degree
                elif highway_shape[0] in ["dual_exit"]:
                    yaw_difference = abs(
                        right_lane_end_wp.transform.rotation.yaw
                    ) - abs(wp.transform.rotation.yaw)
                    already_in_curve = abs(yaw_difference) > degree
                elif highway_shape[0] in ["traffic_light_junction_entry_right"]:
                    for entry_end_wp in entry_wps[1]:
                        if entry_end_wp.road_id == 1450:
                            distance_junction_end = get_distance_junction_end(wp)
                            yaw_difference = abs(
                                entry_end_wp.transform.rotation.yaw
                            ) - abs(
                                entry_end_wp
                                .previous(distance_junction_end)[0]
                                .transform.rotation.yaw
                            )
                            already_in_curve = abs(yaw_difference) < degree
                            break
                elif highway_shape[0] in ["traffic_light_junction_entry_left"]:
                    # 1 entry lane from left with 4 road_ids, starting at the same position and ending on 4 different highway lanes
                    for entry_end_wp in entry_wps[1]:
                        if entry_end_wp.road_id == 1421: # road_id 1421 is middle lane ending on 2nd left lane of highway
                            distance_junction_end = get_distance_junction_end(wp)
                            yaw_difference = abs(
                                entry_end_wp.transform.rotation.yaw
                            ) - abs(
                                entry_end_wp
                                .previous(distance_junction_end)[0]
                                .transform.rotation.yaw
                            )
                            already_in_curve = abs(yaw_difference) < degree
                            break
                else:
                    already_in_curve = False

                # if ego already too far away, dont show in matrix
                if (
                    (direction == "behind")
                    and already_in_curve
                    and (col_distance == 55)
                ):
                    exit_entry_found = True
                    break

                # if wp already in curve, update matrix
                if already_in_curve:
                    if highway_shape[0] in ["traffic_light_junction_entry_left"]:
                        matrix[0][1][z + i] = 0
                        matrix[1][1][z + i] = 0
                    else:
                        matrix[6][1][z + i] = 0
                        matrix[7][1][z + i] = 0
                    col_entryExit = z + i
                    if (
                        z + i + 1 < 8
                    ):  # case: curve is 70m infront (last column in matrix)
                        if highway_shape[0] in ["entry_and_exit"]:
                            if highway_shape[1] == 3:
                                matrix[5][1][z + i + 1] = 3
                            # add entry, if closer than 70m
                            if z + i + 2 < 8:
                                matrix[6][1][z + i + 2] = 0
                                matrix[7][1][z + i + 2] = 0
                        elif highway_shape[0] in ["dual_entry"]:
                            matrix[6][1][z + i + 1] = 0
                            matrix[7][1][z + i + 1] = 0
                        elif highway_shape[0] in ["dual_exit"]:
                            matrix[6][1][z + i + 1] = 0
                            matrix[7][1][z + i + 1] = 0
                    if (z + i - 1 >= 0):
                        if highway_shape[0] in ["traffic_light_junction_entry_right"]:
                            matrix[6][1][z + i - 1] = 0
                            matrix[7][1][z + i - 1] = 0
                    exit_entry_found = True
                    break
    return exit_entry_found, col_entryExit, matrix


In [11]:
def get_highway_shape(wps_grouped, junction, ego_waypoint): #TODO new parameter ego_waypoint     
    # Check if entry or exit
    wps_grouped.sort(key=len)
    # special case for junction 1368: traffic light junction on highway
    if junction.id == 1368:
        if ego_waypoint.road_id == 41:
            highway_type = "traffic_light_junction_entry_left"
        elif ego_waypoint.road_id == 40:
            highway_type = "traffic_light_junction_entry_right"
        else:
            print("Error in get_highway_shape: Junction 1368 (traffic light junction on highway on Map04), ego vehicle not on road 40 or 41")
        straight_lanes = 4
    else:
        if len(wps_grouped[0]) == 2:
            if wps_grouped[0][0][1] != wps_grouped[0][1][1]:
                highway_type = "entry_and_exit"
            elif wps_grouped[0][0][1] == "start":
                highway_type = "dual_entry"
            elif wps_grouped[0][0][1] == "end":
                highway_type = "dual_exit"

        elif len(wps_grouped[0]) == 1:
            if wps_grouped[0][0][1] == "start":
                highway_type = "single_entry"
            else:
                highway_type = "single_exit"
        else:
            print("Junction object w/o shape")
            return ("normal_highway", 4, None, None)

        # Check number of lanes
        road_ids = [wp[0].road_id for wp in wps_grouped[1]]
        if len(set(road_ids)) == 1:
            straight_lanes = 4
        else:
            straight_lanes = 3
# TODO: STRAIGHT LANES UND ENTRY WPS /   EXIT WPS
    entry_wps = None
    exit_wps = None
    # get start & end waypoint of entry / exit as tuple
    if highway_type in ["entry_and_exit", "single_entry", "dual_entry", "traffic_light_junction_entry_left", "traffic_light_junction_entry_right"]:
        entry_wps = get_exit_or_entry(wps_grouped, "entry", junction)
    if highway_type in ["entry_and_exit", "single_exit", "dual_exit"]:
        exit_wps = get_exit_or_entry(wps_grouped, "exit", junction)

    return (highway_type, straight_lanes, entry_wps, exit_wps)

In [12]:
def get_all_lanes(ego_vehicle, ego_wp, junction_waypoints, road_lane_ids, direction_angle):
    """
    Get all lanes related to the junction and the corresponding directions from the ego vehicle's perspective.

    Parameters:
        ego_vehicle (Vehicle): The ego vehicle.
        ego_wp (Waypoint): The waypoint associated with the ego vehicle.
        junction (Junction): The junction of interest.
        road_lane_ids (list): List of road and lane IDs in the world map.
        direction_angle (float): The angle used to determine directions from the ego vehicle.

    Returns:
        dict, list: A dictionary containing lists of lanes for each direction
                    and a list of junction roads with their respective directions and end waypoints.
    """
    ego_location = ego_vehicle.get_location()

    road_id_ego = str(ego_wp.road_id)
    lane_id_ego = ego_wp.lane_id
    start_wps = [[], [], []]
    end_wps = [[], [], [], []]
    closest_start_wp = get_clostest_starting_waypoint(junction_waypoints, ego_location)

    for i, (start_wp, end_wp) in enumerate(junction_waypoints):
        if junction_waypoints[0][0].get_junction().id == 1368 and ((i < 2) or (i>=len(junction_waypoints)-2)):  
            road_id_end_wp = str(end_wp.previous(2)[0].road_id)
            lane_id_end_wp = str(end_wp.previous(2)[0].lane_id)
        else:
            # end
            road_id_end_wp = str(end_wp.next(2)[0].road_id)
            lane_id_end_wp = str(end_wp.next(2)[0].lane_id)
        end_wps[0].append(end_wp)
        end_wps[1].append(road_id_end_wp)
        if road_id_end_wp != road_id_ego:
            end_wps[2].append(
                get_waypoint_direction(
                    ego_vehicle, closest_start_wp, end_wp, direction_angle
                )
            )
        elif road_id_end_wp == road_id_ego:
            end_wps[2].append("ego")
        end_wps[3].append(lane_id_end_wp)

    # get distinct road id + corresponding direction from ego + end_wp
    junction_roads = []
    for i in range(len(end_wps[0])):
        if end_wps[1][i] not in [x[0] for x in junction_roads]:
            junction_roads.append(
                [end_wps[1][i], end_wps[2][i], end_wps[0][i], end_wps[3][i]]
            )

    lanes_all = {
        "ego": [],
        "left": [],
        "straight": [],
        "right": [],
    }  # direction from ego perpective
    for lane_id in road_lane_ids:  # iterate through all lanes of map
        for road in junction_roads:  # for each road of the junction
            if road[0] == lane_id.split("_")[0]:
                if (junction_waypoints[0][0].get_junction().id == 1368) and int(lane_id.split("_")[1])*np.sign(ego_wp.lane_id) < 0 and int(road[0]) != 23:
                    continue
                else:
                    lanes_all[road[1]].append(lane_id.split("_")[1])
    return lanes_all, junction_roads

In [13]:
def getJunctionShape(ego_vehicle, ego_wp, wps, road_lane_ids, direction_angle):
    """
    Determines the shape of the junction in the city matrix and returns relevant information.

    Parameters:
        ego_vehicle (Vehicle): The ego vehicle.
        ego_wp (Waypoint): Waypoint of the ego vehicle.
        junction (Junction): The junction object.
        road_lane_ids (dict): A dictionary containing lane IDs for different directions.
                             Format: {"ego": lane_id, "left": lane_id, "straight": lane_id, "right": lane_id}.
        direction_angle (float): Angle to determine direction at junction.

    Returns:
        tuple: A tuple containing the following information:
            key_value_pairs (list): A list of key-value pairs representing the city matrix.
            lanes_all (dict): A dictionary containing lane information for different directions.
                              Format: {"ego": [lane_ids], "left": [lane_ids], "straight": [lane_ids], "right": [lane_ids]}.
            junction_roads (list): List of junction roads and lane information.
                                   Format: [road_id, direction, outgoing_lane_wp, lane_id].
            yaw (float): The yaw angle of the ego vehicle's rotation.
    """
    lanes_all, junction_roads = get_all_lanes(
        ego_vehicle, ego_wp, wps, road_lane_ids, direction_angle
    )
    key_value_pairs = [
        ("1", [0, 0, 0, 0, 0, 0, 0, 0]),
        ("2", [0, 0, 0, 0, 0, 0, 0, 0]),
        ("3", [0, 0, 0, 0, 0, 0, 0, 0]),
        ("4", [0, 0, 0, 0, 0, 0, 0, 0]),
        ("5", [0, 0, 0, 0, 0, 0, 0, 0]),
        ("6", [0, 0, 0, 0, 0, 0, 0, 0]),
        ("7", [0, 0, 0, 0, 0, 0, 0, 0]),
        ("8", [0, 0, 0, 0, 0, 0, 0, 0]),
    ]
    left_lanes = int((8 - len(lanes_all["ego"])) / 2)
    left_complete = False
    top_lanes = int((8 - len(lanes_all["left"])) / 2)
    top_complete = False
    right_lanes = int((8 - len(lanes_all["straight"])) / 2)
    right_complete = False
    bottom_lanes = int((8 - len(lanes_all["right"])) / 2)
    bottom_complete = False
    i = 0

    while any(
        [not left_complete, not top_complete, not right_complete, not bottom_complete]
    ):

        # 1. ego/left
        if not top_complete:
            for j in range(left_lanes):
                key_value_pairs[j][1][i] = 3

        # 2. top
        if not right_complete:
            for j in range(top_lanes):
                key_value_pairs[i][1][-1 - j] = 3

        # 3. right
        if not bottom_complete:
            for j in range(right_lanes):
                key_value_pairs[-1 - j][1][-1 - i] = 3

        # 4. bottom
        if not left_complete:
            for j in range(bottom_lanes):
                key_value_pairs[-1 - i][1][j] = 3

        # check if sides of matrix are already complete
        i = i + 1
        if i == 8:
            break
        if i == bottom_lanes:
            bottom_complete = True
        if i == right_lanes:
            right_complete = True
        if i == top_lanes:
            top_complete = True
        if i == left_lanes:
            left_complete = True

    ego_transform = ego_vehicle.get_transform()
    ego_rotation = ego_transform.rotation
    yaw = ego_rotation.yaw

    return key_value_pairs, lanes_all, junction_roads, yaw


In [14]:
def detect_surrounding_cars_outside_junction(
    key_value_pairs,
    junction_roads,
    lanes_all,
    ego_vehicle,
    world,
    distance_to_junc,
    junction,
):
    """
    Detects and records surrounding cars outside the junction in the city matrix.

    Parameters:
        key_value_pairs (list): A list of key-value pairs representing the city matrix.
        junction_roads (list): List of junction roads and lane information.
                               Format: [road_id, direction, outgoing_lane_wp, lane_id].
        lanes_all (dict): A dictionary containing lane information for different directions.
                          Format: {"ego": [lane_ids], "left": [lane_ids], "straight": [lane_ids], "right": [lane_ids]}.
        ego_vehicle (Vehicle): The ego vehicle.
        world (CarlaWorld): The world representation.
        distance_to_junc (float): Distance from ego vehicle to the junction in meters.
        junction (Junction): The junction object.

    Returns:
        list: Updated key-value pairs representing the city matrix with information about surrounding cars.
    """
    world_map = world.get_map()
    ego_waypoint = world_map.get_waypoint(ego_vehicle.get_location())
    junction_id = junction.id
    # junction_roads =
    #   [road_id of incoming road, direction from ego-perspective, wp of outgoing lane of incoming road, corresponding lane_id ],
    #       ... for all 4 directions ego, left, right, straight
    #   ]    # Special Highway Traffi

    surrounding_cars = {"ego": [], "left": [], "straight": [], "right": []}
    for actor in world.get_actors():
        if "vehicle" in actor.type_id and actor.id != ego_vehicle.id:
            actor_location = actor.get_location()
            actor_waypoint = world_map.get_waypoint(actor_location)
            if junction_id == 1368 and np.sign(ego_waypoint.lane_id) != np.sign(actor_waypoint.lane_id) and actor_waypoint.road_id in [40, 41]:
                continue
            for road in junction_roads:                
                distance_to_actor = distance(actor_location, road[2].transform.location)
                different_road_distance = None

                if (actor_waypoint.is_junction) and (
                    actor_waypoint.get_junction().id == junction_id
                ):
                    actor_outside_junction = False
                else:
                    actor_outside_junction = True

                if (
                    distance_to_actor <= distance_to_junc * 1.2
                ) and actor_outside_junction:  # add extra 20% because distance_to_actor is calculated with road[2] waypoint, which can be on other lane
                    included = True
                    if str(actor_waypoint.road_id) == road[0]:
                        different_road_distance = None
                    elif (
                        str(actor_waypoint.next(int(distance_to_junc / 2))[0].road_id)
                        == road[0]
                    ):
                        different_road_distance = int(distance_to_junc / 2)
                    elif (
                        str(
                            actor_waypoint.previous(int(distance_to_junc / 2))[
                                0
                            ].road_id
                        )
                        == road[0]
                    ) and (not actor_waypoint.is_junction):
                        different_road_distance = (
                            int(distance_to_junc / 2) * -1
                        )  # negative bc. look back from actor
                    elif (
                        str(actor_waypoint.next(int(distance_to_junc / 3))[0].road_id)
                        == road[0]
                    ):
                        different_road_distance = int(distance_to_junc / 3)
                    elif (
                        str(
                            actor_waypoint.previous(int(distance_to_junc / 3))[
                                0
                            ].road_id
                        )
                        == road[0]
                    ):
                        different_road_distance = (
                            int(distance_to_junc / 3) * -1
                        )  # negative bc. look back from actor
                    else:
                        included = False
                    if included:
                        surrounding_cars[road[1]].append(
                            (actor, different_road_distance)
                        )

    for road in junction_roads:
        for actor, different_road_distance in surrounding_cars[road[1]]:
            actor_location = actor.get_location()
            actor_waypoint = world_map.get_waypoint(actor_location)

            if not (different_road_distance is None):
                if different_road_distance > 0:
                    actor_waypoint_lane_id = actor_waypoint.next(
                        different_road_distance
                    )[0].lane_id
                elif different_road_distance < 0:
                    actor_waypoint_lane_id = actor_waypoint.previous(
                        (different_road_distance * -1)
                    )[0].lane_id
            else:
                actor_waypoint_lane_id = actor_waypoint.lane_id

            # sort list of lane ID's
            if (int(road[3]) < 0 and not junction.id == 1368) or int(road[3]) > 0 and junction.id == 1368:
                lanes_all[road[1]].sort(reverse=True)
            else:
                lanes_all[road[1]].sort()

            # determine lane of actor, i=0: lane most out of lanes incoming the junction
            for i in range(len(lanes_all[road[1]])):
                if lanes_all[road[1]][i] == actor_waypoint_lane_id:
                    break

            # determine number of cells/columns in matrix outside the inner junction
            if (road[1] == "ego") or (road[1] == "straight"):
                if len(lanes_all["left"]) == 0:
                    columns = int((8 - len(lanes_all["right"])) / 2)
                elif len(lanes_all["right"]) == 0:
                    columns = int((8 - len(lanes_all["left"])) / 2)
                else:
                    columns = int(
                        (8 - max([len(lanes_all["left"]), len(lanes_all["right"])])) / 2
                    )  # max number of lanes of right/left
            elif (road[1] == "left") or (road[1] == "right"):
                if len(lanes_all["ego"]) == 0:
                    columns = int((8 - len(lanes_all["straight"])) / 2)
                elif len(lanes_all["straight"]) == 0:
                    columns = int((8 - len(lanes_all["ego"])) / 2)
                else:
                    columns = int(
                        (8 - max([len(lanes_all["ego"]), len(lanes_all["straight"])]))
                        / 2
                    )  # max number of lanes of ego/straight

            # determine cell of actor in matrix
            if ((int(road[3]) > 0) and (actor_waypoint_lane_id > 0)) or (
                (int(road[3]) < 0) and (actor_waypoint_lane_id < 0)
            ):  # actor on outgoing lane
                # location.distance(waypoints[i].transform.location
                actor_distance_junction = actor_waypoint.transform.location.distance(
                    road[2].transform.location
                )
                if actor_distance_junction < (
                    distance_to_junc / columns
                ):  # doesn't matter how many columns: write "2" in the cell clostest to junction inner part
                    c = columns - 1
                elif (columns == 2) or (
                    actor_distance_junction < (distance_to_junc / columns) * 2
                ):  # if 2 columns then it must be the cell farthest away && if 3 columns check if junction is closer than 2/3 of DISTANCE_TO_JUNCTION, then middle cell
                    c = columns - 2
                else:  # if 3 columns and further away, then cell farthest away
                    c = columns - 3
            else:  # incoming actor
                if actor_waypoint.next(int(distance_to_junc / columns))[
                    0
                ].is_junction:  # doesn't matter how many columns: write "2" in the cell clostest to junction inner part
                    c = columns - 1
                elif (columns == 2) or (
                    actor_waypoint.next(int(distance_to_junc / columns) * 2)[
                        0
                    ].is_junction
                ):  # if 2 columns that it must be the cell farsest away && if 3 columns check if junction is closer than 2/3 of DISTANCE_TO_JUNCTION, then middle cell
                    c = columns - 2
                else:  # if 3 columns and further away than 10m, then cell farsest away
                    c = columns - 3

            # write "2" in identified cell
            if road[1] == "ego":
                for j in range(8):
                    # ego road
                    if key_value_pairs[-1 - j][1][c] != 3:
                        # in case ego vehicle is already in that matrix cell
                        if key_value_pairs[-1 - j - i][1][c] == 1:
                            # Calculate dot_product (similarity between the vectors)
                            dot_product = check_car_in_front_or_behind(
                                ego_vehicle.get_location(),
                                actor.get_location(),
                                ego_vehicle.get_transform().rotation,
                            )
                            if dot_product > 0:  # other car infront
                                if (columns != 1) and (
                                    c == columns - 1
                                ):  # already in front cell
                                    key_value_pairs[-1 - j - i][1][c - 1] = 1
                                    key_value_pairs[-1 - j - i][1][c] = 2
                                elif (
                                    columns != 1
                                ):  # not in front cell but more than 1 cells available
                                    key_value_pairs[-1 - j - i][1][c + 1] = 2
                            else:  # other car behind
                                if (columns != 1) and (c == 0):  # already in last cell
                                    key_value_pairs[-1 - j - i][1][c + 1] = 1
                                    key_value_pairs[-1 - j - i][1][c] = 2
                                elif (
                                    columns != 1
                                ):  # not in last cell but more than 1 cells available
                                    key_value_pairs[-1 - j - i][1][c - 1] = 2
                        else:
                            key_value_pairs[-1 - j - i][1][c] = 2
                        break
            elif road[1] == "left":
                for j in range(8):
                    # left road
                    if key_value_pairs[c][1][j] != 3:
                        key_value_pairs[c][1][j + i] = 2
                        break
            elif road[1] == "straight":
                for j in range(8):
                    # straight road
                    if key_value_pairs[j][1][-1 - c] != 3:
                        key_value_pairs[j + i][1][-1 - c] = 2
                        break
            elif road[1] == "right":
                for j in range(8):
                    # right road
                    if key_value_pairs[-1 - c][1][-1 - j] != 3:
                        key_value_pairs[-1 - c][1][-1 - j - i] = 2
                        break

    if junction.id == 1368:
        key_value_pairs[2][1][-3:], key_value_pairs[5][1][-3:] = key_value_pairs[5][1][-3:], key_value_pairs[2][1][-3:]
        key_value_pairs[3][1][-3:], key_value_pairs[4][1][-3:] = key_value_pairs[4][1][-3:], key_value_pairs[3][1][-3:]

    return key_value_pairs


In [15]:
def detect_ego_before_junction(
    key_value_pairs, junction_roads, lanes_all, lane_id_ego, ego_wp, distance_to_junc, junction
):
    """
    Update the city matrix to detect the position of the ego vehicle relative to a junction.

    Parameters:
        key_value_pairs (list of tuples): A list of tuples representing the city matrix.
        junction_roads (list of tuples): A list of tuples containing road IDs and lane IDs of the junction.
        lanes_all (dict): A dictionary containing lane IDs of the roads around the junction.
        lane_id_ego (int): The lane ID of the ego vehicle.
        ego_wp (carla.Waypoint): The waypoint of the ego vehicle.
        distance_to_junc (int): The distance in meters from the ego vehicle to the junction.

    Returns:
        list of tuples: The updated city matrix with the position of the ego vehicle detected.
    """
    # detect ego before junction

    # cast lane id's to int
    for road in junction_roads:
        lanes_all[road[1]] = [int(id) for id in lanes_all[road[1]]]

    # check on which lane ego is: i=0 means most out
    if (lane_id_ego > 0 or (lane_id_ego < 0 and junction.id == 1368)):
        lanes_all["ego"].sort(reverse=True)
    else:
        lanes_all["ego"].sort()
    for i in range(int(len(lanes_all["ego"]))):
        if lanes_all["ego"][i] == lane_id_ego:
            break  # i=0: lane most out

    # check how many lanes the roads right and left of ego have
    if len(lanes_all["left"]) == 0:
        columns = int((8 - len(lanes_all["right"])) / 2)
    elif len(lanes_all["right"]) == 0:
        columns = int((8 - len(lanes_all["left"])) / 2)
    else:
        columns = int(
            (8 - min([len(lanes_all["left"]), len(lanes_all["right"])])) / 2
        )  # min number of lanes of right/left

    # determine cell of ego in matrix
    if ego_wp.next(int(distance_to_junc / columns))[
        0
    ].is_junction:  # doesn't matter how many columns: write "1" in the cell clostest to junction inner part
        c = columns - 1
    elif columns == 2:  # if 2 columns that it must be the cell farsest away
        c = columns - 2
    elif ego_wp.next(int(distance_to_junc / columns) * 2)[
        0
    ].is_junction:  # if 3 columns check if junction is closer than 10m, then middle cell
        c = columns - 2
    else:  # if 3 columns and further away than 10m, then cell farsest away
        c = columns - 3

    for j in range(8):
        if key_value_pairs[-1 - j][1][c] != 3:
            key_value_pairs[-1 - j - i][1][c] = 1
            break

    return key_value_pairs

In [16]:
def divide_bounding_box_into_boxes(junction_waypoints):
    """
    Divide the bounding box of the given junction into smaller boxes.

    Parameters:
        junction (carla.Junction): A carla.Junction object.

    Returns:
        list: A list of boxes, where each box is represented as a list containing a carla.BoundingBox object and an integer value initialized to 0.
    """
    junction_waypoints = list(sum(junction_waypoints, ()))
    grouped_waypoints = group_waypoints(junction_waypoints)
    locations = []
    for group in grouped_waypoints:
        temp = []
        for wp in group:
            temp.append((wp.transform.location.x, wp.transform.location.y))
            temp = list(dict.fromkeys(temp))
        locations.append(temp)
    locations = [item for row in locations for item in row]

    x_values = []
    y_values = []
    for point in locations:
        x_values.append(point[0])
        y_values.append(point[1])

    center_values_x = find_center_points(x_values)
    center_values_y = find_center_points(y_values)

    center_values_y.sort(key=lambda x: x[0], reverse=True)
    points = list(itertools.product(center_values_x, center_values_y))

    boxes = []
    for point in points:
        box = [
            carla.BoundingBox(
                carla.Location(point[0][0], point[1][0], 0),
                carla.Vector3D(point[0][1], point[1][1], 100),
            ),
            0,
        ]
        boxes.append(box)

    return boxes

In [17]:
def fill_boxes(boxes, ego_vehicle, world, junction_waypoints):
    """
    Fill the boxes with information about surrounding cars.

    Parameters:
        boxes (list): A list of boxes, where each box is represented as a list containing a carla.BoundingBox object and an integer value initialized to 0.
        ego_vehicle (carla.Vehicle): The ego vehicle, a carla.Vehicle object.
        world (carla.World): The carla.World object.

    Returns:
        list: A list of boxes with updated integer values representing surrounding cars:
              0 - No car inside the box
              1 - The ego vehicle is inside the box
              2 - Another surrounding car is inside the box
    """
    special_junction_row = None
    surrounding_cars = []
    for actor in world.get_actors():
        if "vehicle" in actor.type_id:
            distance_to_actor = distance(
                actor.get_location(), ego_vehicle.get_location()
            )
            if distance_to_actor <= 100:
                surrounding_cars.append(actor)
    for car in surrounding_cars:
        car_location = car.get_location()
        if junction_waypoints[-1][0].get_junction().id == 1368 and distance(junction_waypoints[-1][0].transform.location, car_location) < 30 and world.get_map().get_waypoint(car_location).is_junction:
            special_junction_row = 1 
            continue
        for box in boxes:
            if car.id == ego_vehicle.id:
                if box[0].contains(car_location, carla.Transform()):
                    box[1] = 1
            else:
                if box[0].contains(car_location, carla.Transform()):
                    box[1] = 2

    return boxes, special_junction_row

In [18]:
def detect_cars_inside_junction(key_value_pairs, ego_vehicle, junction_waypoints, yaw, world):
    """
    Detect and mark cars inside the junction in the grid representation.

    Parameters:
        key_value_pairs (list of tuples): A list of tuples containing lane IDs and their
                                         respective grid representations. The grid
                                         representations are 2D lists where each element
                                         represents the state of the cell.
        ego_vehicle (carla.Vehicle): The ego vehicle for reference.
        junction (carla.Junction): The carla.Junction object representing the junction.
        yaw (float): The yaw angle (rotation) of the ego vehicle in degrees.
        world (carla.World): The carla.World object representing the simulation world.

    Returns:
        dict: A dictionary with updated lane IDs and their corresponding grid representations
              after marking cars inside the junction.

    Notes:
        - The function uses grid representations to identify cars inside the junction.
        - It calculates the corners of the junction to determine the junction boundaries.
        - The `rotate_grid` function is used to handle the rotation of the junction if needed.
        - The function checks whether rows and columns should be flipped based on the ego vehicle's orientation.
        - The `divide_bounding_box_into_boxes`, `fill_boxes`, and `build_grid` functions are used to construct the grid.
    """
    grid_corners = get_grid_corners(key_value_pairs)

    rows_min = min([value[0] for value in grid_corners])
    rows_max = max([value[0] for value in grid_corners])
    cols_min = min([value[1] for value in grid_corners])
    cols_max = max([value[1] for value in grid_corners])

    rows = rows_max - rows_min + 1
    columns = cols_max - cols_min + 1

    if check_flipping_rows_and_columns(ego_vehicle):
        rows, columns = columns, rows
        
    boxes = divide_bounding_box_into_boxes(junction_waypoints)

    boxes, special_junction_row = fill_boxes(boxes, ego_vehicle, world, junction_waypoints)

    grid = build_grid(boxes)

    grid = rotate_grid(grid, yaw)

    i = rows_min
    for row in grid:
        j = cols_min
        for cell in row:
            key_value_pairs[i][1][j] = cell[1]
            j = j + 1
        i = i + 1
    if special_junction_row is not None:
        key_value_pairs[special_junction_row][1][4] = 2

    return dict(key_value_pairs)

# Game Loop

In [20]:
same_junction = False
on_junction = False
junctions_detected = False
junction_old = None
wps_old = None
highway_shape_old = None
highway_shape = None
wrong_shape = None
junction_shape = None
matrix = None
junction_id = None
street_type = None 
road_lane_ids = get_all_road_lane_ids(world_map = world.get_map())
on_entry = False
junction_id_skip = None
df = initialize_dataframe()
#try:
while True:
    #print("----------------------------------")
    ego_location = ego_vehicle.get_location()
    ego_waypoint = world_map.get_waypoint(ego_location)
    ego_on_highway = check_ego_on_highway(ego_location, road_lane_ids, world_map)
    if ego_on_highway:
        distance_to_junc = 80
    else:
        distance_to_junc = 20
    

    if junctions_detected and not ego_waypoint.is_junction:
        junctions_detected = False 
        same_junction = False
        
    elif junctions_detected:
        pass
    elif ego_waypoint.is_junction and on_junction == False:
        on_junction = True
    elif on_junction:
        junction_old = junction
        wps_old = wps
        highway_shape_old = highway_shape
        on_junction = False
        junctions_detected = True
    elif is_junction_ahead(ego_waypoint, distance_to_junc) and not on_junction: #and  != get_junction_ahead(ego_waypoint, distance_to_junc).id:
        junction = get_junction_ahead(ego_waypoint, distance_to_junc)
        junction_id = junction.id
        highway_junction = is_highway_junction(ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle)


    # CASES
    # 1. On city junction
    if (ego_waypoint.is_junction and not highway_junction and junction.id != 459) or (junction.id == 1368 and ego_waypoint.is_junction):
        street_type = "On junction"
        print("On Junction")
        if junction_shape:
            matrix = detect_surrounding_cars_outside_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )
            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, wps, yaw, world
            )
            for key, value in matrix.items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")
    
    # 2. City junction ahead
    elif (is_junction_ahead(ego_waypoint, distance_to_junc) and not highway_junction and junction.id != 459) or (junction.id == 1368 and is_junction_ahead(ego_waypoint, distance_to_junc)):
        if junction.id == 1368:
            street_type = "Highway traffic light"
            junction = get_junction_ahead(ego_waypoint, distance_to_junc)
            wps = junction.get_waypoints(carla.LaneType().Driving)
            starting_wp = get_clostest_starting_waypoint(wps, ego_location)
            wps = remove_wps(starting_wp, wps)

            print("removed wp")
            for wp in wps:
                print(wp[0], wp[1])

        else:
            street_type = "Junction ahead"
            wps = get_junction_ahead(ego_waypoint, distance_to_junc)


        
        junction_shape, lanes_all_junction, junction_roads_junction, yaw, = getJunctionShape(
            ego_vehicle, ego_waypoint, wps, road_lane_ids, direction_angle
        )
        if junction_shape != "Error":
            print("Junction ahead")
            matrix = detect_ego_before_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_waypoint.lane_id,
                ego_waypoint,
                distance_to_junc,
                junction
            )
            
            matrix = detect_surrounding_cars_outside_junction(
                matrix,
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )

            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, wps, yaw, world
            )

            for key, value in dict(matrix).items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")



    # 3. Highway entry/exit in front or behind
    elif (is_junction_ahead(ego_waypoint, 80) or (is_junction_behind(ego_waypoint, 40) and street_type == "Highway with entry/exit")) and not on_entry and highway_junction and ego_on_highway and junction_id_skip != junction.id:
        street_type = "Highway with entry/exit"
        #print("Highway Junction detected")
        if not same_junction and is_junction_ahead(ego_waypoint, 80):
            same_junction = True
            # print("selbst erkennnen")
            junction = get_junction_ahead(ego_waypoint, 80)

            wps = junction.get_waypoints(carla.LaneType().Driving)
            closest_waypoint = get_clostest_starting_waypoint(wps, ego_location)
            wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
            # # print grouped wps
            # for group in wps:
            #     print("######################### NEW GROUP #########################")
            #     for wp in group:
            #         print("-------------------")
            #         print(wp[0], wp[1])
            highway_shape = get_highway_shape(wps, junction, ego_waypoint) # TODO: New parameter ego_waypoint
            if not highway_shape is None:
                print(highway_shape[0], highway_shape[1])

        if (highway_shape is None) and (not junction_old is None):
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old
            wrong_shape = True

        matrix = create_city_matrix(ego_location, road_lane_ids, world_map)            
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            if not highway_shape is None and (highway_shape[0] != "normal_highway" or wrong_shape):
                matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape, wps, matrix, ego_waypoint, junction, cars_on_entryExit, direction_angle) 
                if not junction_old is None and junction_old.id != junction.id:
                    matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape_old, wps_old, matrix, ego_waypoint, junction_old, cars_on_entryExit, direction_angle, is_2nd_update=True) 
        for key, value in matrix.items():
            print(value)
        print("----------------------------------------------------------------------------------------------------------------------")


    # 4. Highway entry / exit
    elif is_junction_ahead(ego_waypoint, 20)  and highway_junction and street_type != "Highway with entry/exit":
        print("On highway entry")
        street_type = "On highway entry"
        on_entry = True
        junction = get_junction_ahead(ego_waypoint, 20)
        junction_id_skip = junction.id

        wps = junction.get_waypoints(carla.LaneType().Driving)
        closest_waypoint, middle_location = get_waypoint_on_highway_junction(ego_waypoint, ego_location, wps, world_map)
        wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
        highway_shape = get_highway_shape(wps, junction, ego_waypoint) # TODO: New parameter ego_waypoint

        on_entry = True
        if (highway_shape is None) and (not junction_old is None):
            same_junction = False
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old


        matrix = create_city_matrix(middle_location, road_lane_ids, world_map, ghost=True)
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                middle_location, ego_vehicle, matrix, road_lane_ids, world, radius, True, highway_shape, ghost=True
            )

            if not highway_shape is None:
                matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape, wps, matrix, closest_waypoint, junction, cars_on_entryExit, direction_angle, ghost=True, on_entry=on_entry) 
                # if not junction_old is None and junction_old.id != junction.id:
                #     matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape_old, wps_old, matrix, closest_waypoint, junction_old, cars_on_entryExit, True, on_entry) 
                
        for key, value in matrix.items():
            print(value)
        print("----------")

    # 5. Normal Road
    else:
        # # test ###################
        # break
        on_junction = False
        junction_old = None
        on_entry = False
        print("normal road")
        same_junction = False
        if ego_on_highway:
            street_type = "On highway"
        else:
            street_type = "Non highway street"
        matrix = create_city_matrix(ego_location, road_lane_ids, world_map)
        if matrix:
            matrix, _ = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            for key, value in matrix.items():
                print(value)


    

    steering_angle = get_steering_angle(ego_vehicle)
    speed = get_speed(ego_vehicle)
    print("============================================================")
    if matrix:
        row_data = get_row(matrix)
        row_data["speed"] = speed
        row_data["steering_angle"] = steering_angle
        row_data["street_type"] = street_type
        # df = df.append(row_data, ignore_index=True)

    # clock.tick_busy_loop(60)
    time.sleep(0.3)
    world.tick()

    
"""
except Error:
    pass

finally:
    df.to_csv("x_hours.csv")
 1401"""

single_entry 4
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 0, 3]
[3, 3, 3, 3, 3, 3, 0, 3]
----------------------------------------------------------------------------------------------------------------------
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 0, 3]
[3, 3, 3, 3, 3, 3, 0, 3]
----------------------------------------------------------------------------------------------------------------------
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 0, 3]
[3, 3, 3, 3, 3, 3, 0, 3]
----------------------------------------------------------------------------------------------------------------------
[3, 3, 3, 3, 3, 3, 3, 3]
[3,

KeyboardInterrupt: 

# Tests

In [ ]:
# TEST: Print grouped wps in junction - current variable content
for group in wps:
    for wp in group:
        
        if wp[0]:
            print(wp[0].__str__(), wp[1])
            print(wp[0].road_id)
            print(wp[0].lane_id)
            #print("yaw:", wp[0].get_forward_vector())
    print("----------------------------------------------------------------------------------------------------------------------")

In [ ]:
# TEST: generate grouped wps in junction & highway shape
ego_location = ego_vehicle.get_location()
ego_waypoint = world_map.get_waypoint(ego_location)
ego_waypoint.is_junction
junction = ego_waypoint.get_junction()
junction = get_junction_ahead(ego_waypoint, 80)
wps = junction.get_waypoints(carla.LaneType().Driving)

closest_waypoint = get_clostest_starting_waypoint(wps, ego_location)
print(closest_waypoint)
#wps = prepare_waypoints(closest_waypoint, wps, ego_location)
#get_highway_shape(wps, junction, ego_waypoint)

In [ ]:
wps[0][0].get_junction()

In [ ]:
# TEST: get ego road/lane id loop
while True:
    time.sleep(0.3)
    ego_location = ego_vehicle.get_location()
    ego_waypoint = world_map.get_waypoint(ego_location)
    print("#####################")
    #print(is_junction_ahead(ego_waypoint, 80))
    print(ego_waypoint.road_id)
    print(ego_waypoint.lane_id)
    #print(ego_waypoint.is_junction)

In [ ]:
# TEST: get ego information single moment
ego_location = ego_vehicle.get_location()
ego_waypoint = world_map.get_waypoint(ego_location)
# print(ego_waypoint.road_id)
print(is_junction_ahead(ego_waypoint, 80))
#is_junction_behind(ego_waypoint, 40)
# print(ego_waypoint.transform.location)

In [ ]:
get_junction_ahead(ego_waypoint, 80).id

In [ ]:
# TEST: get second vehicle information single moment
second_vehicle_location = second_vehicle.get_location()
second_vehicle_waypoint = world_map.get_waypoint(second_vehicle_location)
print(second_vehicle_waypoint.road_id)